In [9]:
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU
import glob
import matplotlib.pyplot as plt

In [6]:
# Import image paths
imagesRaw = glob.glob("./new_images/*.jpg")
imagesRaw.sort()

In [ ]:
imageDataset = tf.data.Dataset.from_tensor_slices(images)
image

In [ ]:
# Create Datasets
dataset_train = tf.data.Dataset.from_tensor_slices((train_images, train_classes))
dataset_test = tf.data.Dataset.from_tensor_slices((test_images, test_classes))
dataset_validation = tf.data.Dataset.from_tensor_slices((validation_images, validation_classes))

In [1]:
def load_jpeg(image):
    # Load the image turn into jpeg
    decoded = tf.io.read_file(image)
    imageTf = tf.image.decode_jpeg(decoded)
    # Normalize the images
    returnImage = tf.cast(imageTf, tf.float32) / 255.0
    return returnImage

In [ ]:
imagesTfImages = imagesTf.map(load_jpeg)

In [4]:
# Define the constants
S = 3 # Divide the image to have S*S cells
C = 1 # Number of classes 
B = 2 # Number of bounding boxes to be predicted per cell

In [19]:
model = tf.keras.Sequential([
    #First Layer
    tf.keras.layers.Conv2D(64, (7,7), strides=(2, 2), activation=LeakyReLU(alpha=0.1),  input_shape=(488,488,3)), #padding=3?
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Second Layer
    tf.keras.layers.Conv2D(192, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Third Layer
    tf.keras.layers.Conv2D(128, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(256, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(256, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(512, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Fourth Layer
    # +++ Repeated block
    tf.keras.layers.Conv2D(256, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(512, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    
    tf.keras.layers.Conv2D(256, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(512, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),

    tf.keras.layers.Conv2D(256, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(512, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),

    tf.keras.layers.Conv2D(256, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(512, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    # +++ END BLOCK
    tf.keras.layers.Conv2D(512, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Fifth layer
    # +++ Repeated Block
    tf.keras.layers.Conv2D(512, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    
    tf.keras.layers.Conv2D(512, (1,1), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    # +++ END BLOCK
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(1024, (3,3), strides=(2, 2), activation=LeakyReLU(alpha=0.1), padding="same"),
    
    #Sixth Layer
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    tf.keras.layers.Conv2D(1024, (3,3), activation=LeakyReLU(alpha=0.1), padding="same"),
    
    # Final Output Layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, input_shape=(1024 * S * S,), activation=LeakyReLU(alpha=0.1)),
    tf.keras.layers.Dense(S * S * (B*5+C), input_shape=(4096,), activation=None)
])

The yoloLoss function is an adpation from here: https://blog.emmanuelcaradec.com/humble-yolo-implementation-in-keras/. All credit goes to Emmanuel Caradec

In [23]:
def yoloLoss(y_true, y_pred):
    print(y_true)
    grid = [ [[float(x),float(y)]]*B for y in range(S) for x in range(S)]

    pred_boxes = tf.reshape(y_pred[...,3:], (-1,S*S,B,5))
    true_boxes = tf.reshape(y_true[...,3:], (-1,S*S,B,5))
    y_true_conf = true_boxes[...,4]

    y_pred_xy   = pred_boxes[...,0:2] + tf.Variable(grid)
    y_true_xy   = true_boxes[...,0:2]

    xy_loss    = tf.math.reduce_sum(tf.math.reduce_sum(tf.math.square(y_true_xy - y_pred_xy),axis=-1)*y_true_conf, axis=-1)
    
    y_pred_wh   = pred_boxes[...,2:4]
    y_true_wh   = pred_boxes[...,2:4]
    y_true_conf = true_boxes[...,4]

    wh_loss    = tf.math.reduce_sum(tf.math.reduce_sum(tf.math.square(tf.math.sqrt(y_true_wh) - tf.math.sqrt(y_pred_wh)), axis=-1)*y_true_conf, axis=-1)
                                    
    y_true_class = y_true[...,0:2]
    y_pred_class = y_pred[...,0:2]
    y_true_conf = true_boxes[...,4]

    clss_loss  = tf.math.reduce_sum(tf.math.square(y_true_class - y_pred_class)*y_true_conf, axis=-1)
                                    
    intersect_wh = tf.math.maximum(tf.zeros_like(y_pred_wh), (y_pred_wh + y_true_wh)/2 - tf.math.abs(y_pred_xy - y_true_xy) )
    intersect_area = intersect_wh[...,0] * intersect_wh[...,1]
    true_area = y_true_wh[...,0] * y_true_wh[...,1]
    pred_area = y_pred_wh[...,0] * y_pred_wh[...,1]
    union_area = pred_area + true_area - intersect_area
    iou = intersect_area / union_area

    conf_loss = tf.math.reduce_sum(tf.math.square(y_true_conf*iou - y_pred_conf)*y_true_conf, axis=-1)
    
    loss =  clss_loss + xy_loss + wh_loss + conf_loss
                                    
    return loss
